In [1]:
!pip install -q selenium undetected-chromedriver beautifulsoup4 webdriver-manager

In [2]:
GAMEBOY_LP_URL = 'https://longplays.org/infusions/longplays/longplays.php?cat_id=30'

In [2]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def setup_browser():
    options = uc.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    
    driver = uc.Chrome(options=options)
    return driver

In [3]:
from bs4 import BeautifulSoup
from typing import Optional

class Scraper:
    def __init__(self):
        self.driver = setup_browser()
        self.wait = WebDriverWait(self.driver, 20)
    
    def load_page(self, url: str) -> bool:
        try:
            self.driver.get(url)
            # Wait for Cloudflare challenge to pass (if any)
            time.sleep(5)
            return True
        except Exception as e:
            print(f"Error loading page: {e}")
            return False
    
    def wait_for_element(self, selector: str) -> Optional[str]:
        try:
            element = self.wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, selector))
            )
            return element
        except Exception:
            return None

    def parse_content(self, selector: str = "body") -> Optional[dict]:
        try:
            # Wait for content to load
            self.wait_for_element(selector)
            
            # Get the page source after JavaScript execution
            html = self.driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            # Example parsing (modify based on your needs)
            data = {
                'title': soup.title.text if soup.title else None,
                'content': soup.select(selector)[0].text if soup.select(selector) else None
            }
            return data
        except Exception as e:
            print(f"Error parsing content: {e}")
            return None

        

In [ ]:
def main():
    scraper = DynamicScraper()
    
    # Example usage
    url = "https://example.com"
    if scraper.load_page(url):
        # Wait for specific content (modify selector as needed)
        data = scraper.parse_content("#main-content")
        
        if data:
            print(f"Title: {data['title']}")
            print(f"Content: {data['content'][:200]}...")  # First 200 chars
    
    # Clean up
    scraper.driver.quit()

In [ ]:
main()

In [40]:
detail_table = soup.find_all('table', class_='tblDetail')
links = []
for element in detail_table:
    links = element.find_all('a', href=True)
# download_cells = detail_table.find_all('a', href=True)

In [47]:
for link in links:
    if link['']
    # link_url = link['href']
    # if '../../' in link_url:
    #     link_url = urljoin(url, link_url)
    
    # print(link_url)

https://longplays.org/infusions/longplays/longplays.php?cat_id=30&author=8923
https://longplays.org/infusions/longplays/longplays.php?cat_id=30
https://longplays.org/infusions/longplays/longplays.php?file_id=771
https://youtu.be/cR-l6VbPQOE
https://www.mobygames.com/game/5335/kirbys-dream-land/
